# Build a classification model : 

### 1- Load the libraries:

In [67]:
import pandas as pd
from sklearn import ensemble
import numpy as np 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn import ensemble
import matplotlib.pyplot as plt
from sklearn import model_selection 
from sklearn.tree import plot_tree 

### 2- Load data:

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv')

In [6]:
df

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25546,2017-12-10,0.00,49,34,False
25547,2017-12-11,0.00,49,29,False
25548,2017-12-12,0.00,46,32,False
25549,2017-12-13,0.00,48,34,False


In [170]:
numrows=25549
weather=pd.DataFrame({'yesterday':[0.0]*numrows,
                             'today':[0.0]*numrows,
                             'tomorrow':[True]*numrows,
                            })

seq = ['yesterday','today','tomorrow']

weather=weather.reindex(columns=seq)

for z in range(numrows):
    i = z + 2
    yesterday = df.iloc[(i-2),1]
    today = df.iloc[(i-1),1]
    tomorrow = df.iloc[i,1]
   
    
    weather.iat[z,0] = yesterday
    weather.iat[z,1] = today
    weather.iat[z,2] = tomorrow 

In [171]:
weather

,yesterday,today,tomorrow
0,0.47,0.59,True
1,0.59,0.42,True
2,0.42,0.31,True
3,0.31,0.17,True
4,0.17,0.44,True
...,...,...,...
25544,0.00,0.00,False
25545,0.00,0.00,False
25546,0.00,0.00,False
25547,0.00,0.00,False


In [172]:
weather=weather[~weather.isin([np.nan, np.inf, -np.inf]).any(1)] 

In [173]:
weather.describe()

,yesterday,today
count,25544.000000,25544.000000
mean,0.106238,0.106220
std,0.239046,0.239037
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.100000,0.100000
max,5.020000,5.020000


In [174]:
weather.shape

(25544, 3)

### 3- Spliting Data :

In [175]:
X = weather.drop('tomorrow', axis=1).copy()
X.head()

,yesterday,today
0,0.47,0.59
1,0.59,0.42
2,0.42,0.31
3,0.31,0.17
4,0.17,0.44


In [176]:
y = weather['tomorrow'].copy()
y.head()

0    True
1    True
2    True
3    True
4    True
Name: tomorrow, dtype: bool

In [177]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2, random_state=0)

### 3- Scaleing the data:

In [178]:
from sklearn.preprocessing import StandardScaler

In [179]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

### 4- Modeling the Data: 

In [180]:
clf = ensemble.RandomForestClassifier( n_estimators=20).fit(X, y)

In [181]:
y_pred=clf.predict(X_test)

In [182]:
y_pred

array([False, False, False, ..., False, False, False])

In [183]:
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score

In [184]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[2558  408]
 [1229  914]]
              precision    recall  f1-score   support

       False       0.68      0.86      0.76      2966
        True       0.69      0.43      0.53      2143

    accuracy                           0.68      5109
   macro avg       0.68      0.64      0.64      5109
weighted avg       0.68      0.68      0.66      5109

0.6795850459972598


In [185]:
#modifyign the data to work with the model
x = weather[['today', 'yesterday']].values
y = weather.tomorrow.values
x = x.reshape(weather.shape[0], 2)

### 5: Bulding the tree:

In [186]:
clf2=DecisionTreeClassifier(random_state=42)
clf2= clf2.fit(X_train, y_train)

In [187]:
path = clf2.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
ccp_alphas = ccp_alphas[:1]

clff = []
for ccp_alpha in ccp_alphas:
    clf2 = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf2.fit(X_train, y_train)
    clff.append(clf2)

In [188]:
Rain = weather[(weather['today'] > 0.50)
                                & 
                                (weather['yesterday'] < 0.20)]['today']


In [152]:
X_test

array([[-0.4541176 ,  0.24339664],
       [-0.4541176 , -0.43481088],
       [-0.4541176 , -0.43481088],
       ...,
       [-0.32354825, -0.43481088],
       [-0.4541176 , -0.43481088],
       [ 0.89509904, -0.43481088]])

In [198]:
pruned = DecisionTreeClassifier(random_state=42, 
                                       ccp_alpha=Rain)
pruned = pruned.fit(X_train, y_test) 
pruned

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [196]:
np.any(X_test)

True

In [197]:
from sklearn import tree
plt.figure(figsize=(15,7.5))
plot_tree(pruned, label='all', filled=True)

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

<Figure size 1080x540 with 0 Axes>